## Challenge code
 
 In this challenge you will be given the following functions: 
 - `create_Hamiltonian`: In which you build the Transverse Ising Hamiltonian for $N=4$ and a magnetic field intensity `h`. **You must complete this function.**
 - `model`: This QNode builds a general enough ansatz for the ground state. This circuit must depend on some parameters `params`, which you will later optimize. It returns the expectation value of the Hamiltonian for the output state of the circuit. **You must complete this function.**
 - `train`: This function returns the parameters that minimize the output of `model`. **You must complete this function.**
 
![img](images/spaceship_4.png)

 ### Input
 
 As input to this problem, you are given:
 
 - `h` (`float`): Magnetic field intensity applied to the spin chain.
  
 ### Output
 
 This code will output a `float` corresponding to the energy of the ground state.
 
 If your solution matches the correct one within the given tolerance specified in `check` (in this case it's an relative tolerance of `0.1`), the output will be `"Correct!"` Otherwise, you will receive a `"Wrong answer"` prompt.
 
 Good luck!
 ### Imports
 The cell below specifies the libraries you should use in this challenge. Run the cell to import the libraries. ***Do not modify the cell.***

In [1]:
import json
import pennylane as qml
from pennylane import numpy as np

### Code
 Complete the code below. Note that during QHack, some sections were not editable. We've marked those sections accordingly here, but you can still edit them if you wish.

In [7]:
num_qubits = 4
def create_Hamiltonian(h):
    """
    Function in charge of generating the Hamiltonian of the statement.

    Args:
        h (float): magnetic field strength

    Returns:
        (qml.Hamiltonian): Hamiltonian of the statement associated to h
    """
    coeffs1 = num_qubits*[-1]
    # (wire+1) % num_qubits do the closed loop magic
    obs1 = [qml.PauliZ(wire) @ qml.PauliZ((wire+1) % num_qubits) for wire in range(num_qubits)]
    
    coeffs2 = num_qubits*[-h]
    obs2 = [qml.PauliX(wire) for wire in range(num_qubits)]

    coeffs = coeffs1 + coeffs2
    obs = obs1 + obs2

    # qml.Hamiltonian is just a shortcut for linear combination
    return qml.Hamiltonian(coeffs, obs)


dev = qml.device("default.qubit", wires=num_qubits)
@qml.qnode(dev)
def model(params, H):
    """
    To implement VQE you need an ansatz for the candidate ground state!
    Define here the VQE ansatz in terms of some parameters (params) that
    create the candidate ground state. These parameters will
    be optimized later.

    Args:
        params (numpy.array): parameters to be used in the variational circuit
        H (qml.Hamiltonian): Hamiltonian used to calculate the expected value

    Returns:
        (float): Expected value with respect to the Hamiltonian H
    """
    qml.BasicEntanglerLayers(weights=params[0], wires=range(num_qubits), rotation=qml.RX)
    qml.BasicEntanglerLayers(weights=params[1], wires=range(num_qubits), rotation=qml.RY)
    qml.BasicEntanglerLayers(weights=params[2], wires=range(num_qubits), rotation=qml.RZ)
    return qml.expval(H)


def train(h):
    """
    In this function you must design a subroutine that returns the
    parameters that best approximate the ground state.

    Args:
        h (float): magnetic field strength

    Returns:
        (numpy.array): parameters that best approximate the ground state.
    """
    # gradient descent algorithm
    opt = qml.GradientDescentOptimizer(stepsize=0.1)
    max_iterations = 1000
    conv_tolerance = 1e-4

    Hamiltonian = create_Hamiltonian(h)
    # 3 sets of variable, for 3 layers, for each qubit (4)
    params = np.random.rand(3, 3, num_qubits, requires_grad=True)

    for n in range(max_iterations):
        params, prev_cost = opt.step_and_cost(model, params, H=Hamiltonian)
        new_cost = model(params, Hamiltonian)

        if n % 50 == 0:
            print(f"Iteration {n}: cost = {new_cost:.7f}")

        if abs(new_cost - prev_cost) < conv_tolerance:
            print(f"Converged at iteration {n}")
            break

    return params

inp = 1.0
params = train(inp)

Iteration 0: cost = -2.6318009
Iteration 50: cost = -4.9059074
Iteration 100: cost = -4.9840945
Iteration 150: cost = -5.0740465
Iteration 200: cost = -5.1896975
Iteration 250: cost = -5.2060574
Converged at iteration 264


These functions are responsible for testing the solution. You will need to run the cell below. ***Do not modify the cell.***

In [8]:
def run(test_case_input: str) -> str:
    ins = json.loads(test_case_input)
    params = train(ins)
    return str(model(params, create_Hamiltonian(ins)))


def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-1
    ), "The expected value is not correct."

### Test cases
 Running the cell below will load the test cases. ***Do not modify the cell***.
 - input: 1.0
 	+ expected output: -5.226251859505506
 - input: 2.3
 	+ expected output: -9.66382463698038
 - input: 0.5
 	+ expected output: -4.271558410139714
 - input: 1.5
 	+ expected output: -6.760008550556145

In [9]:
test_cases = [['1.0', '-5.226251859505506'], ['2.3', '-9.66382463698038'], ['0.5', '-4.271558410139714'], ['1.5', '-6.760008550556145']]

### Solution testing
 Once you have run every cell above, including the one with your code, the cell below will test your solution. Run the cell. If you are correct for all of the test cases, it means your solutions is correct. Otherwise, you need to double check your work. ***Do not modify the cell below.***

In [ ]:
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '1.0'...
Iteration 0: cost = -1.4317228
Iteration 50: cost = -4.9133308
Iteration 100: cost = -5.0934982
Iteration 150: cost = -5.1488610
Iteration 200: cost = -5.1783663
Iteration 250: cost = -5.1981208
Iteration 300: cost = -5.2099124
Iteration 350: cost = -5.2166993
Converged at iteration 352
Correct!
Running test case 1 with input '2.3'...
Iteration 0: cost = -2.7092314
Iteration 50: cost = -9.6091559
Iteration 100: cost = -9.6578072
Converged at iteration 116
Correct!
Running test case 2 with input '0.5'...
Iteration 0: cost = -1.5429696
Iteration 50: cost = -4.0339331
Iteration 100: cost = -4.0645979
Iteration 150: cost = -4.0976955
Iteration 200: cost = -4.1255738
Iteration 250: cost = -4.1452355
Iteration 300: cost = -4.1573050
Iteration 350: cost = -4.1643477
Converged at iteration 356
Correct!
Running test case 3 with input '1.5'...
Iteration 0: cost = -2.8173753
Iteration 50: cost = -6.5227366
Iteration 100: cost = -6.7142083
Iteration 150: cos